<a href="https://colab.research.google.com/github/davidbonar1/03MAIR_Algoritmos_de_Optimizacion_2019/blob/master/David_Bo%C3%B1ar_alvarez_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad Guiada 2 de Algoritmos de Optimización

David Boñar

https://colab.research.google.com/drive/1axnNKuTviMRG6FLv5c74Ss97F_CStxht

https://github.com/davidbonar1/03MAIR_Algoritmos_de_Optimizacion_2019

**Ejercicio 1: Viaje por el río. Encontrar la ruta óptima para viajar de un embarcadero a otro con el menor coste posible.**

Considerando una tabla T(i,j) para almacenar todos los precios que ofrencen los embarcaderos.

Si no es posible ir desde i a j, daremos un valor alto para garantizar que ese trayecto no se va a elegir en la ruta óptima.

Establecer una tabla intermedia para las soluciones óptimas parciales para ir desde i a j.

In [0]:
TARIFAS = [
[0,5,4,3,999,999,999],
[999,0,999,2,3,999,11],
[999,999, 0,1,999,4,10],
[999,999,999, 0,5,6,9],
[999,999, 999,999,0,999,4],
[999,999, 999,999,999,0,3],
[999,999,999,999,999,999,0]
]

def precios(Tarifas):
  #Número total de nodos
  N = len(Tarifas[0])

  #Inicialización de la tabla de precios con valores muy altos
  precios = [ [9999]*N for i in [9999]*N]
  ruta = [ [""]*N for i in [""]*N] #en la matriz de la ruta guardamos también 
                                    # los nodos intermedios
  
  for i in range(N-1): #se recorren tas las filas menos el de destino
    for j in range(i+1, N): #se recorren todas las columnas menos la primera
      minimo = Tarifas[i][j]
      ruta[i][j] = i
      
      for k in range(i, j):
        if precios[i][k] + Tarifas[k][j] < minimo:
            minimo = min(minimo, precios[i][k] + Tarifas[k][j] )
            ruta[i][j] = k
        precios[i][j] = minimo
        
  return precios,ruta

PRECIOS, RUTA = precios (TARIFAS)

In [3]:


def calcular_ruta(RUTA, desde, hasta):
  if desde == hasta:
    #print("Ir a :" + str(desde))
    return desde 
  else:
    return str(calcular_ruta(RUTA, desde, RUTA[desde][hasta])) +  ',' + str(RUTA[desde][hasta]) 

print("\nLa ruta es:")  
calcular_ruta(RUTA, 0,4) 


La ruta es:


'0,0,1'

Problema 2: Ramificación y poda. Asignación de tareas:

Dada una tabla con rendimientos y trabajamos, se debe maximizar el rendimiento en cuanto a la asignación N tareas a N agentes, pudiendo asginar cada tarea a un solo agente.

In [0]:
COSTES=[[11,12,18,40],
        [14,15,13,22],
        [11,17,19,23],
        [17,14,20,28]]

In [0]:
#Definimos primero las funciones necesarias

#Coste inferior para soluciones parciales
#  (1,3,) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def CI(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion del coste límite inferior
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += min([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR


#Coste superior
def CS(S,COSTES):
  VALOR = 0
  #Valores establecidos
  for i in range(len(S)):
    VALOR += COSTES[i][S[i]]

  #Estimacion
  for i in range(len(COSTES)):
    if i not in S:
      VALOR += max([COSTES[j][i] for j in range(len(S), len(COSTES))])
  return VALOR

#Genera los nodos siguientes (tantos hijos como como posibilidades haya para la siguiente elemento de la tupla)
#(0,) -> (0,1), (0,2), (0,3)
def crear_hijos(NODO, N):
  HIJOS = []
  for i in range(N):
    if i not in NODO:
      HIJOS.append({'s':NODO +(i,)})
  return HIJOS

def valor(S,COSTES):
  VALOR = 0
  for i in range(len(S)):
    VALOR +=COSTES[S[i]][i]
    return VALOR

 

In [4]:
#Proceso principal

def ramificacion_y_poda(COSTES):
#Construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas).
#Nodos del grafo  { s:(1,2),CI:3,CS:5  }
  #print(COSTES)
  #Se inicializa con una solución cualquiera
  DIMENSION = len(COSTES)  
  MEJOR_SOLUCION=tuple(i for i in range(len(COSTES)))
  CotaSup = valor(MEJOR_SOLUCION,COSTES)
  #print("Cota Superior:", CotaSup)

  #Guardamos los nodos que se deben explorar (ramificación)
  NODOS=[]
  NODOS.append({'s':(), 'ci':CI((),COSTES)} )

  iteracion = 0

  while( len(NODOS) > 0):
    iteracion +=1

    nodo_prometedor = [min(NODOS, key=lambda x:x['ci']) ][0]['s'] #de entre los nodos que faltan por explorar, se selecciona el más prometedor
    #print("Nodo prometedor:", nodo_prometedor)

    #Ramificacion
    #Se generan los hijos
    HIJOS =[ {'s':x['s'], 'ci':CI(x['s'], COSTES)   } for x in crear_hijos(nodo_prometedor, DIMENSION)]

    #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
    NODO_FINAL = [x for x in HIJOS if len(x['s']) == DIMENSION ]
    if len(NODO_FINAL ) >0: 
      #print("\n********Soluciones:",  [x for x in HIJOS if len(x['s']) == DIMENSION] )
      if NODO_FINAL[0]['ci'] < CotaSup: #se compara la cota inferior con la superior y nos quedamos con el nodo
        CotaSup = NODO_FINAL[0]['ci']
        MEJOR_SOLUCION = NODO_FINAL

    #Poda
    HIJOS = [x for x in HIJOS if x['ci'] < CotaSup]

    #Añadimos los hijos para ser analizados
    NODOS.extend(HIJOS) 

    #Eliminamos el nodo ramificado
    NODOS =  [  x for x in NODOS if x['s'] != nodo_prometedor]
    
    print("La solucion final es:" ,MEJOR_SOLUCION , " en " , iteracion , " iteraciones" , " para dimension: " ,DIMENSION)
    


ramificacion_y_poda(COSTES)

La solucion final es: (0, 1, 2, 3)  en  1  iteraciones  para dimension:  4


- ¿Qué complejidad tiene el algoritmo por fuerza bruta? ---> n!
- Generar matrices con valores aleatorios de mayores dimensiones (5,6,7,...) y ejecutar ambos algoritmos. ¿A partir de qué dimensión el algoritmo por fuerza bruta deja de ser una opción? --> 11x11
- ¿Hay algún valor de la dimensión a partir del cuál ramificación y poda deja de ser una opción? (Demasiado tiempo de resolución)

In [4]:
from itertools import permutations

#Algoritmo por fuerza bruta
def fuerza_bruta(COSTES):
  mejor_valor = 10e10
  mejor_solucion = ()

  for s in list(permutations(range(len(COSTES)))):
    valor_tmp = valor(s,COSTES)
    if valor_tmp < mejor_valor:
      mejor_valor = valor_tmp
      mejor_solucion = s

  print("La mejor solución es:", mejor_solucion, " con valor: ",mejor_valor)

fuerza_bruta(COSTES)

La mejor solución es: (0, 1, 2, 3)  con valor:  11


In [0]:
#Generar las matrices mayores con números aleatorios entre 0 y 100
import random

def matriz_aleatoria(dimension):
  matriz = [[0 for i in range(dimension)] for j in range(dimension)]
  for i in range(len(matriz)):
      for j in range(len(matriz[0])):
          matriz[i][j] = random.randint(0, 100)
  return matriz

In [8]:
# matriz de rango 11

matriz_11 = matriz_aleatoria(11)

print(matriz_11)

[[27, 22, 10, 15, 97, 78, 35, 10, 23, 81, 93], [92, 63, 72, 87, 16, 96, 78, 97, 12, 95, 5], [32, 59, 80, 89, 63, 82, 83, 84, 52, 33, 38], [41, 13, 54, 49, 41, 87, 55, 50, 19, 99, 42], [92, 13, 68, 64, 26, 51, 20, 21, 22, 85, 91], [11, 31, 89, 49, 47, 83, 47, 47, 28, 69, 98], [68, 59, 77, 14, 64, 20, 83, 90, 0, 37, 64], [97, 76, 2, 16, 38, 76, 76, 21, 82, 1, 65], [9, 98, 91, 57, 46, 96, 71, 42, 88, 86, 89], [62, 23, 8, 82, 96, 80, 79, 71, 89, 84, 40], [52, 77, 89, 80, 54, 59, 95, 30, 3, 35, 22]]


In [10]:
#Medimos el tiempo de ejecución para fuerza bruta y ramificación y poda
%timeit fuerza_bruta(matriz_11)

La mejor solución es: (3, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10)  con valor:  5
La mejor solución es: (3, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10)  con valor:  5
La mejor solución es: (3, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10)  con valor:  5
La mejor solución es: (3, 0, 1, 2, 4, 5, 6, 7, 8, 9, 10)  con valor:  5
1 loop, best of 3: 25.4 s per loop


In [11]:
%%timeit
ramificacion_y_poda(matriz_11)

La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es: (0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10)  en  1  iteraciones  para dimension:  11
La solucion final es

Como se puede ver, con matrices de 11x11 la diferencia en los tiempos de ejecución es abismal. Para fuerza bruta 25 segundos y para ramificación y poda 1,14 milisegundos

In [0]:
#Ahora probaremos hasta que valor la fuerza bruta deja de ser opcion:

#Para una matriz de 30x30 ya no se puede (fallo en memoria ram, demasiada)
#Para una matriz de 20x20 ocurre el mismo fallo (fallo en memoria ram, demasiada)
#Hemos ido bajando y en todos (17x17, 15x15, 12x12) tenemos el mismo problema, por lo que 11x11 sería el orden máximo para ejecutar este algoritmo
matriz_11 = matriz_aleatoria(11)


In [9]:
fuerza_bruta(matriz_11)

La mejor solución es: (8, 0, 1, 2, 3, 4, 5, 6, 7, 9, 10)  con valor:  9


Ahora probaremos hasta cuando merece la pena ramificación y poda:

Con una matriz de 50x50 el tiempo de ejecución es de 11,2 ms, muy aceptable todavía.

Con una de 150x150 el tiempo pasa a ser de 1 segundo aproximadamente.

Con una matriz de 3000x3000, la técnica de ramificación y poda tarda 1 hora por iteración, por lo que el tiempo de ejecución es demasiado como para poder utilizarlo.

In [0]:
matriz_150 = matriz_aleatoria(150)

In [0]:
%%timeit
ramificacion_y_poda(matriz_150)